# 1. Imports and dataset


In [157]:
from sklearn import datasets, metrics
import numpy as np

In [158]:
digits = datasets.load_digits()
num_split = int(0.7 * len(digits.data))
train_features = digits.data[:num_split]  # train data (features)
train_labels = digits.target[:num_split]  # train labels (y-values)
test_features = digits.data[num_split:]  # test data (features)
test_labels = digits.target[num_split:]  # test labels(y-values)

In [159]:
# Normalizing the train dataset to values between 0 and 1
for i in range(len(train_features)):
    for j in range(len(train_features[i])):
        train_features[i][j] /=16

# 2. EM-algorithm to find a Gaussian NBC
- Assume conditional independence (covariance = variance for the attribute)
- Normalize the data to avoid multiplications with very small values in the likelihoods
- You can use the overall change in cluster centers between two iterations as a stop criterion. Minimal movement: assume convergence
- Make sure dimensions of covariance matrix are correct, possible to only calculate the diagonal

In [343]:
class EM_:
    
    def __init__(self, X, K):
        
        # Initialize prior probs, means and covariances
        self.prior = np.ones(K)/K
        self.means = np.zeros([K, len(X[0])])
        self.cov = np.ones([K, len(X[0])])*0.1
        self.r = np.zeros([len(X), K])

        # Creating subsets
        subset = []
        start = 0
        end = int(len(X))/K
        for i in range(K):
            subset.append(X[start:int((i+1)*end)])
            start = int((i+1)*end)
        
        # Calculating first means
        for k,subs in enumerate(subset):
            N = len(subs)
            for j, image in enumerate(subs):
                for h, pixel in enumerate(image):
                    self.means[k][h] +=pixel/N 

                
        # Initiating covariances (variances here)
        for i,subs in enumerate(subset):
            N = len(subs)
            for j, image in enumerate(subs):
                for h, pixel in enumerate(image):
                    self.cov[i][h] += ((pixel-self.means[i][h])**2)/N
    
    
        
    def fit(self, X, K):
        diff = 10
        nr_it = 0
        mean_last_step = self.means.copy()
        while nr_it<10: # Will change to diff>0.1 or similar
            nr_it +=1
                
            # E-step
            EM_.E_step_(self,X,K,nr_it)
                
            # M-step
            EM_.M_step_(self,X, K)
                
            diff  = np.linalg.norm(mean_last_step-self.means)
            mean_last_step = self.means.copy()
            print('Iteration: ' , nr_it, 'Diff: ', diff)
                
            
        return
                
        
    def E_step_(self,X,K, nr_it):
        cov = self.cov
        means = self.means
        prior = self.prior

        if nr_it > 14:
            print('cov', cov)
            print('means', means)
            print('prior', prior)
        # Iterating through all images and for each image we iterate through all classes
        for i,image in enumerate(X):

            expected = np.zeros(K) 
        
            for k in range(K):
                # Calculating the bessel probability P(xi|ø)
                prob = 1
                
                for p, pixel in enumerate(image):
                    prob *= 1/(np.sqrt(2*np.pi*cov[k][p])) * np.exp(-(pixel-means[k][p])**2/(2*cov[k][p]))
                    
                        
                expected[k] = prior[k]*prob 
                
            ri = expected/sum(expected)
            self.r[i] = ri 

            
            
            
        return
        
    def M_step_(self, X, K):
            
        # Computing r_k
        r_k = np.zeros([K])
        for i, image in enumerate(self.r):
            for k, prob in enumerate(image):
                r_k[k] += prob
    
        # Computing new prior probabilities
        self.prior = r_k / len(X)
        
        # Updating means
        for k in range(K):
            mean_k = 0
            for i, image in enumerate(X):
                mean_k += self.r[i][k] * image
            self.means[k] = mean_k/r_k[k]
        
        # Updating covariances
        for k in range(K):
            cov_k = np.ones([len(X[0])])*0.1 
            for i, image in enumerate(X):
                cov_k += self.r[i][k] * image**2
            self.cov[k] = cov_k/r_k[k] - self.means[k]**2
            
        
            
        return
    
    
    def predict_(self,X):
        predictions = []
        prior = self.prior
        cov = self.cov
        means = self.means
        
        # Iterating through all images
        for image in X:
            probability_values = np.zeros(len(prior))
        
            # Iterating through all possible classes with their prior probabilities and multiply probabilities corresponding to the class 
            for k,prior_prob in enumerate(prior):
                probability_values[k] = prior_prob
            
                # Iterating through the image's pixels
                for p, pixel in enumerate(image):
                    probability_values[k] *= 1/(np.sqrt(2*np.pi*cov[k][p])) * np.exp(-(pixel-means[k][p])**2/(2*cov[k][p]))
    
            # Appending the class with highest probability
            predictions.append(np.argmax(probability_values))

        return predictions
            
        

In [344]:
EM = EM_(train_features, 10)

In [345]:
EM.fit(train_features,10)

Iteration:  1 Diff:  1.210593924688377
Iteration:  2 Diff:  1.8830501693258805
Iteration:  3 Diff:  2.085428849197478
Iteration:  4 Diff:  1.26708945986675
Iteration:  5 Diff:  3.021430573825891


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:105: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:112: RuntimeWarning: divide by zero encountered in true_divide


Iteration:  6 Diff:  nan
Iteration:  7 Diff:  nan
Iteration:  8 Diff:  nan
Iteration:  9 Diff:  nan
Iteration:  10 Diff:  nan


In [326]:
pred = EM.predict_(train_features)

# 3. Clustering



In [224]:
predictions = predict(X, prior, means, cov)

# 4. k-means

In [120]:
from sklearn.cluster import KMeans

In [150]:
KM = KMeans(n_clusters=10)
clusters = KM.fit(train_features)
predictions_KMeans = KM.predict(train_features)

# 5. Comparison

In [152]:
metrics.confusion_matrix(train_labels,predictions_KMeans)

array([[  0,   0,   0,   0,   0,   0,   0, 125,   0,   0],
       [  2,   0,   0,   0,  25,   0,   0,   0,  39,  63],
       [  0,   3,   0,   1, 106,   0,   3,   0,   1,  10],
       [  0,   2,   2,  16,   0,   0, 110,   0,   0,   0],
       [  0,   7,   0,   0,   0, 111,   0,   0,   4,   2],
       [  1,   0,  94,  29,   0,   1,   1,   0,   0,   0],
       [124,   0,   0,   0,   0,   0,   0,   1,   0,   2],
       [  0, 124,   0,   0,   0,   0,   0,   0,   1,   0],
       [  1,   1,   4,  39,   2,   0,   1,   0,   4,  70],
       [  0,   6,   2,  97,   0,   0,   3,   0,  15,   2]])

In [225]:
metrics.confusion_matrix(train_labels,predictions)

array([[125,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [129,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [124,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [130,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [124,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [126,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [127,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [125,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [122,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [125,   0,   0,   0,   0,   0,   0,   0,   0,   0]])